# United States - Crime Rates - 1960 - 2014

### Introduction:

This time you will create a data

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b538873fec51916fb60d654243bba6c6ffb6704eabd2331e8e423a48d0d4b2ec
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import expr, col, mean, when, sum, count, desc, min, max
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv).

In [3]:
!wget -O US_Crime_Rates.csv https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv

--2024-04-11 09:52:18--  https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5052 (4.9K) [text/plain]
Saving to: ‘US_Crime_Rates.csv’

US_Crime_Rates.csv  100%[===================>]   4.93K  --.-KB/s    in 0s      

2024-04-11 09:52:18 (26.8 MB/s) - ‘US_Crime_Rates.csv’ saved [5052/5052]



### Step 3. Assign it to a variable called crime.

In [4]:
crime = spark.read.csv('US_Crime_Rates.csv', sep=',', header=True, inferSchema=True)

### Step 4. What is the type of the columns?

In [5]:
crime.dtypes

[('Year', 'int'),
 ('Population', 'int'),
 ('Total', 'int'),
 ('Violent', 'int'),
 ('Property', 'int'),
 ('Murder', 'int'),
 ('Forcible_Rape', 'int'),
 ('Robbery', 'int'),
 ('Aggravated_assault', 'int'),
 ('Burglary', 'int'),
 ('Larceny_Theft', 'int'),
 ('Vehicle_Theft', 'int')]

In [6]:
crime.show(10)

+----+----------+-------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Year|Population|  Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----+----------+-------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960| 179323175|3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961| 182992000|3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962| 185771000|3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963| 188483000|4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964| 191141000|4564600| 364220| 4200400|  9360|        21420| 130390|            203050| 1213200|     

##### Have you noticed that the type of Year is int64. But pandas has a different type to work with Time Series. Let's see it now.

### Step 5. Convert the type of the column Year to datetime64

In [11]:
import pyspark.sql.functions as F

In [15]:
crime = crime.withColumn('Year', F.to_date(col('Year'), 'yyyy'))

In [16]:
crime.show(5)

+----------+----------+-------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|      Year|Population|  Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----------+----------+-------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960-01-01| 179323175|3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961-01-01| 182992000|3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962-01-01| 185771000|3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963-01-01| 188483000|4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964-01-01| 191141000|4564600| 364220| 4200400|  9360|       

In [17]:
crime.dtypes

[('Year', 'date'),
 ('Population', 'int'),
 ('Total', 'int'),
 ('Violent', 'int'),
 ('Property', 'int'),
 ('Murder', 'int'),
 ('Forcible_Rape', 'int'),
 ('Robbery', 'int'),
 ('Aggravated_assault', 'int'),
 ('Burglary', 'int'),
 ('Larceny_Theft', 'int'),
 ('Vehicle_Theft', 'int')]

### Step 6. Set the Year column as the index of the dataframe

### Step 7. Delete the Total column

In [18]:
crime = crime.drop('Total')

In [20]:
assert 'Total' not in crime.columns
crime.columns

['Year',
 'Population',
 'Violent',
 'Property',
 'Murder',
 'Forcible_Rape',
 'Robbery',
 'Aggravated_assault',
 'Burglary',
 'Larceny_Theft',
 'Vehicle_Theft']

### Step 8. Group the year by decades and sum the values

#### Pay attention to the Population column number, summing this column is a mistake

In [30]:
crime = crime.withColumn('decade', F.floor(F.year(col('Year')) / 10)*10)

In [43]:
col_for_sum = [c for c in crime.columns if c not in ['Population', 'Year', 'decade']]
col_for_sum

['Violent',
 'Property',
 'Murder',
 'Forcible_Rape',
 'Robbery',
 'Aggravated_assault',
 'Burglary',
 'Larceny_Theft',
 'Vehicle_Theft']

In [44]:
agg_dict = {c: "sum" for c in col_for_sum}
agg_dict['Population'] = "max"

In [62]:
crime_per_dec = crime.groupBy('decade').agg(agg_dict).orderBy('decade')

In [63]:
crime_per_dec.show()

+------+------------------+-----------------------+-----------+---------------+----------+-------------+------------------+-----------+------------+------------+-------------+------------------+
|decade|max(Larceny_Theft)|max(Aggravated_assault)|max(decade)|max(Population)| max(Year)|max(Burglary)|max(Vehicle_Theft)|max(Murder)|max(Robbery)|max(Violent)|max(Property)|max(Forcible_Rape)|
+------+------------------+-----------------------+-----------+---------------+----------+-------------+------------------+-----------+------------+------------+-------------+------------------+
|  1960|           3888600|                 311090|       1960|      201385000|1969-01-01|      1981900|            878500|      14760|      298850|      661870|      6749000|             37170|
|  1970|           6601000|                 629480|       1970|      220099000|1979-01-01|      3327700|           1112800|      21460|      480700|     1208030|     11041500|             76390|
|  1980|           787240

# OR

In [65]:
agg_expr = [F.sum(c).alias(c) for c in col_for_sum]
agg_expr.append(F.max('Population').alias('Population'))

In [66]:
crime_per_dec = crime.groupBy('decade').agg(*agg_expr).orderBy('decade')

In [67]:
crime_per_dec.show()

+------+--------+---------+------+-------------+-------+------------------+--------+-------------+-------------+----------+
|decade| Violent| Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|Population|
+------+--------+---------+------+-------------+-------+------------------+--------+-------------+-------------+----------+
|  1960| 4134930| 45160900|106180|       236720|1633510|           2158520|13321100|     26547700|      5292100| 201385000|
|  1970| 9607930| 91383800|192230|       554570|4159020|           4702120|28486000|     53157800|      9739900| 220099000|
|  1980|14074328|117048900|206439|       865639|5383109|           7619130|33073494|     72040253|     11935411| 248239000|
|  1990|17527048|119053499|211664|       998827|5748930|          10568963|26750015|     77679366|     14624418| 272690813|
|  2000|13968056|100944369|163068|       922499|4230366|           8652124|21565176|     67970291|     11412834| 307006550|
|  2010|

### Step 9. What is the most dangerous decade to live in the US?

In [71]:
agg_dict = {col: "max" for col in crime_per_dec.columns if col !='decade'}

result = crime_per_dec.agg(agg_dict)

result.show()

+------------------+-----------------------+---------------+-------------+------------------+-----------+------------+------------+-------------+------------------+
|max(Larceny_Theft)|max(Aggravated_assault)|max(Population)|max(Burglary)|max(Vehicle_Theft)|max(Murder)|max(Robbery)|max(Violent)|max(Property)|max(Forcible_Rape)|
+------------------+-----------------------+---------------+-------------+------------------+-----------+------------+------------+-------------+------------------+
|          77679366|               10568963|      318857056|     33073494|          14624418|     211664|     5748930|    17527048|    119053499|            998827|
+------------------+-----------------------+---------------+-------------+------------------+-----------+------------+------------+-------------+------------------+



# Solution is this, but it does not perform well

In [80]:
columns = crime_per_dec.columns
columns.remove("decade")

max_values = []

for col in columns:
    max_value = crime_per_dec.groupBy("decade").agg(F.max(col).alias(col))
    max_decade = max_value.orderBy(F.desc(col)).first()
    max_values.append((max_decade["decade"], max_decade[col], col))

print(max_values)

[(1990, 17527048, 'Violent'), (1990, 119053499, 'Property'), (1990, 211664, 'Murder'), (1990, 998827, 'Forcible_Rape'), (1990, 5748930, 'Robbery'), (1990, 10568963, 'Aggravated_assault'), (1980, 33073494, 'Burglary'), (1990, 77679366, 'Larceny_Theft'), (1990, 14624418, 'Vehicle_Theft'), (2010, 318857056, 'Population')]


In [81]:
for decade, val, col_name in max_values:
  print(decade, val, col_name)

1990 17527048 Violent
1990 119053499 Property
1990 211664 Murder
1990 998827 Forcible_Rape
1990 5748930 Robbery
1990 10568963 Aggravated_assault
1980 33073494 Burglary
1990 77679366 Larceny_Theft
1990 14624418 Vehicle_Theft
2010 318857056 Population
